<a href="https://colab.research.google.com/github/wajidminhas/AI-DREAMER/blob/main/streaming/streaming_agent_sdk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q openai-agents

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.3/130.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:00


In [2]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
from agents import Agent, Runner, OpenAIChatCompletionsModel, set_default_openai_client, set_default_openai_api, set_tracing_disabled, AsyncOpenAI
from google.colab import userdata
from agents.run import RunConfig
import os

set_tracing_disabled(True)

gemini_api_key = userdata.get("GEMINI_API_KEY")

if not gemini_api_key:
  raise ValueError("GEMINI_API_KEY not set")

external_client = AsyncOpenAI(
    api_key = gemini_api_key,
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
)

model = OpenAIChatCompletionsModel(
    openai_client = external_client,
    model= "gemini-2.0-flash"
)



#now creating streaming event in agents sdk

In [4]:
import asyncio

# from openai.types.responses import ResponseTextDeltaEvent
# from agents import Agent, Runner

async def joke_and_fun():
    agent : Agent = Agent(
        name= "Joke & Fun",
        instructions="You are a Helplful Assitent and a Joke & fun actor",
        model= model
    )

    result = Runner.run_streamed(agent, input= "give five joke as fun")
    print(type(result), result ,"\n\n")
    async for event in result.stream_events():
        print(event)

asyncio.run(joke_and_fun())

<class 'agents.result.RunResultStreaming'> RunResultStreaming:
- Current agent: Agent(name="Joke & Fun", ...)
- Current turn: 0
- Max turns: 10
- Is complete: False
- Final output (NoneType):
    None
- 0 new item(s)
- 0 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResultStreaming` for more details) 


AgentUpdatedStreamEvent(new_agent=Agent(name='Joke & Fun', instructions='You are a Helplful Assitent and a Joke & fun actor', handoff_description=None, handoffs=[], model=<agents.models.openai_chatcompletions.OpenAIChatCompletionsModel object at 0x7dfbdba07b90>, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], outpu

#this will output the text generated by the LLM token-by-token.

In [6]:
import asyncio
from openai.types.responses import ResponseTextDeltaEvent
from agents import Agent, Runner

async def main():
    agent = Agent(
        name="Joker",
        instructions="You are a helpful assistant.",
        model=model
    )

    result = Runner.run_streamed(agent, input="Please tell me 5 jokes.")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)


if __name__ == "__main__":
    asyncio.run(main())

Alright, here are 5 jokes for you:

1.  Why don't scientists trust atoms?
    Because they make up everything!

2.  Why did the scarecrow win an award?
    Because he was outstanding in his field!

3.  What do you call a lazy kangaroo?
    Pouch potato!

4.  Why did the bicycle fall over?
    Because it was two tired!

5.  Parallel lines have so much in common.
    It's a shame they'll never meet.


#this will ignore raw events and stream updates to the user.

In [9]:
import asyncio
import random
from agents import Agent, ItemHelpers, Runner, function_tool

@function_tool
def how_many_jokes() -> int:
    return random.randint(1, 10)


async def main():
    agent = Agent(
        name="Joke & Fun",
        instructions="First call the `how_many_jokes` tool, then tell that many jokes.",
        tools=[how_many_jokes],
        model=model
    )

    result = Runner.run_streamed(
        agent,
        input="Hello",
    )
    print("=== Run starting ===")

    async for event in result.stream_events():
        # We'll ignore the raw responses event deltas
        if event.type == "raw_response_event":
            continue
        # When the agent updates, print that
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
            continue
        # When items are generated, print them
        elif event.type == "run_item_stream_event":
            if event.item.type == "tool_call_item":
                print("-- Tool was called")
            elif event.item.type == "tool_call_output_item":
                print(f"-- Tool output: {event.item.output}")
            elif event.item.type == "message_output_item":
                print(f"-- Message output:\n {ItemHelpers.text_message_output(event.item)}")
            else:
                pass  # Ignore other event types

    print("=== Run complete ===")


if __name__ == "__main__":
    asyncio.run(main())

=== Run starting ===
Agent updated: Joke & Fun
-- Tool was called
-- Tool output: 7
-- Message output:
 Ok, I will tell you 7 jokes.

Why don’t scientists trust atoms?

Because they make up everything!

Why did the scarecrow win an award?

Because he was outstanding in his field!

What do you call a lazy kangaroo?

Pouch potato!

Why did the bicycle fall over?

Because it was two tired!

Why did the tomato turn red?

Because it saw the salad dressing!

What do you call a bear with no teeth?

A gummy bear!

Why did the teddy bear say no to dessert?

Because she was stuffed!

=== Run complete ===
